In [1]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.5.3


In [2]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.3.3

**Import the classes**

In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.evaluators.Evaluators


In [4]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


In [5]:
import java.io.Serializable

import java.io.Serializable


Instantiate a SparkSession

In [6]:
val conf = new SparkConf().setMaster("local[*]").setAppName("Taxi-cust-density")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@2afc1856

### Schema class
Let us create a case class to describe the schema for the data:

In [7]:

case class Taxi_prediction(

    pickup_lat:String,
    pickup_long:String,
    date_col:String,
    day_of_the_week:String,
    OnHour:Int,
    cust_density:Int
    
)

defined class Taxi_prediction


#### Feature Engineering

We then define the set of raw features that we would like to extract from the data. The 
raw features are defined using [FeatureBuilders](https://docs.transmogrif.ai/Developer-Guide#featurebuilders), 
and are strongly typed. TransmogrifAI supports the following basic feature types: `Text`, 
`Numeric`, `Vector`, `List` , `Set`, `Map`. In addition it supports many specific feature 
types which extend these base types: Email extends Text; Integral, Real and Binary extend 
Numeric; Currency and Percentage extend Real. For a complete view of the types supported 
see the Type Hierarchy and Automatic Feature Engineering section in the Documentation.

Basic `FeatureBuilders` will be created for you if you use the TransmogrifAI CLI to bootstrap 
your project as described here. However, it is often useful to edit this code to customize 
feature generation and take full advantage of the Feature types available (selecting the 
appropriate type will improve automatic feature engineering steps).
When defining raw features, specify the extract logic to be applied to the raw data, and 
also annotate the features as either predictor or response variables via the FeatureBuilders:


In [8]:
import org.apache.spark.sql.{Encoders}
implicit val srEncoder = Encoders.product[Taxi_prediction]

class[pickup_lat[0]: string, pickup_long[0]: string, date_col[0]: string, day_of_the_week[0]: string, OnHour[0]: int, cust_density[0]: int]

In [9]:
// val tpep_pickup_datetime = FeatureBuilder.Text[Taxi].extract(_.tpep_pickup_datetime.toText).asPredictor
// val tpep_dropoff_datetime = FeatureBuilder.Text[Taxi].extract(_.tpep_dropoff_datetime.toText).asPredictor

null

In [10]:
object Extractors extends Serializable {
    class pickup_lat_extractor extends Function[Taxi_prediction,Text] with Serializable {
      def apply(i:Taxi_prediction): Text = i.pickup_lat.toText
   }
    class pickup_long_extractor extends Function[Taxi_prediction,Text] with Serializable {
      def apply(o:Taxi_prediction): Text = o.pickup_long.toText
   }
    
    class date_col_extractor extends Function[Taxi_prediction,Text] with Serializable {
      def apply(o:Taxi_prediction): Text = o.date_col.toText
   }
     class day_of_the_week_extractor extends Function[Taxi_prediction,Text] with Serializable {
      def apply(o:Taxi_prediction): Text = o.day_of_the_week.toText
   }
     class OnHour_extractor extends Function[Taxi_prediction,Real] with Serializable {
      def apply(o:Taxi_prediction): Real = o.OnHour.toReal
   }
     class cust_density_extractor extends Function[Taxi_prediction,RealNN] with Serializable {
      def apply(o:Taxi_prediction): RealNN = o.cust_density.toRealNN
   }
    
//     class total_amount_extractor extends Function[Taxi,RealNN] with Serializable {
//       def apply(o:Taxi): RealNN = o.total_amount.toRealNN
//    }
    
    //add more features here if needed....
       
}

$line31.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Extractors$@25c00897

In [11]:
// val passenger_count = FeatureBuilder.Integral[Taxi].extract(_.passenger_count.toIntegral).asPredictor

null

In [12]:
// val trip_distance = FeatureBuilder.Real[Taxi].extract(_.trip_distance.toReal).asPredictor

null

In [13]:
// val pickup_long = FeatureBuilder.Text[Taxi].extract(_.pickup_long.toText).asPredictor
// val pickup_lat = FeatureBuilder.Text[Taxi].extract(_.pickup_lat.toText).asPredictor

null

In [14]:
import Extractors._

val pickup_lat = FeatureBuilder.Text[Taxi_prediction].extract(new pickup_lat_extractor).asPredictor
val pickup_long = FeatureBuilder.Text[Taxi_prediction].extract(new pickup_long_extractor).asPredictor
val date_col  =    FeatureBuilder.Text[Taxi_prediction].extract(new date_col_extractor).asPredictor
val day_of_the_week= FeatureBuilder.Text[Taxi_prediction].extract(new day_of_the_week_extractor).asPredictor
val OnHour = FeatureBuilder.Real[Taxi_prediction].extract(new OnHour_extractor).asPredictor
val cust_density= FeatureBuilder.RealNN[Taxi_prediction].extract(new cust_density_extractor).asResponse




$line31.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Extractors$@25c00897

In [15]:
// val dropoff_long= FeatureBuilder.Text[Taxi].extract(_.dropoff_long.toText).asPredictor
// val dropoff_lat = FeatureBuilder.Text[Taxi].extract(_.dropoff_lat.toText).asPredictor

null

In [16]:
// val total_amount = FeatureBuilder.RealNN[Taxi].extract(_.total_amount.toRealNN).asResponse

null

In [17]:
//  val total_amount1 =total_amount.fillMissingWithMean().zNormalize()

null

In [18]:
 val trainFilePath = "/home/whirldata/Documents/moving/half_Data.csv"

/home/whirldata/Documents/moving/half_Data.csv

Create a training data reader from the `trainFilePath` using `DataReaders.Simple`

In [19]:
val trainDataReader = DataReaders.Simple.csvCase[Taxi_prediction](
      path = Option(trainFilePath)
    )

com.salesforce.op.readers.CSVProductReader@4b9df32d

### Create a feature sequence and transmogrify it

`.transmogrify()` is a Transmografai shortcut to many estimators. This is in essence the automatic feature engineering Stage of TransmogrifAI. This stage can be discarded in favor of hand-tuned feature engineering and manual vector creation followed by combination using the VectorsCombiner Transformer (short-hand Seq(....).combine()) if the user desires to have complete control over feature engineering.

The next stage applies another powerful transmogrifai Estimator — the SanityChecker. The SanityChecker applies a variety of statistical tests to the data based on Feature types and discards predictors that are indicative of label leakage or that show little to no predictive power. This is in essence the automatic feature selection Stage of TransmogrifAI:

In [20]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val features = Seq(pickup_lat,pickup_long,date_col,day_of_the_week,OnHour).transmogrify()
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_00000000000a

In [21]:
// val label = total_amount1
// val checkedFeatures = label.sanityCheck(features, removeBadFeatures = true)

null

In [22]:
println(features)

### Model selector
Create a prediction(model) based on RegressionModelSelector. We are using Gradient Boosted Trees and Random Forest. Notice how input is applied  of `salesPrice` and `features`.

In [23]:
import com.salesforce.op.stages.impl.regression.RegressionModelSelector
import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpGBTRegressor, OpDecisionTreeRegressor}

val prediction1 = RegressionModelSelector
      .withCrossValidation(
        dataSplitter = Some(splitter), seed = randomSeed,
        modelTypesToUse = Seq(OpGBTRegressor, OpDecisionTreeRegressor)
      ).setInput(cust_density,features).getOutput()

Feature(name = OnHour-cust_density-date_col-day_of_the_week-pickup_lat-pickup_long_4-stagesApplied_Prediction_000000000013, uid = Prediction_000000000013, isResponse = true, originStage = ModelSelector_000000000013, parents = [RealNN_000000000006,OPVector_000000000009], distributions = [])

Create an evaluator of type Regression and call setLabelCol and setPredictionCol

In [24]:
val evaluator = Evaluators.Regression().setLabelCol(cust_density).setPredictionCol(prediction1)

OpRegressionEvaluator_000000000014

### Workflow  and WorkflowModel

Workflow for TransmogrifAI. Takes the final features that the user wants to generate as 
inputs and constructs the full DAG needed to generate them from those features lineage. 
Then fits any estimators in the pipeline dag to create a sequence of transformations that 
are saved in a workflow model.
When we now call `train` on this workflow, it automatically computes and executes the 
entire DAG of Stages needed to compute the features fitting all the estimators on the training data in the process. 
Calling score on the fitted workflow then transforms the underlying training data to 
produce a DataFrame with the all the features manifested. The score method can optionally 
be passed an evaluator that produces metrics.
`workflow.train()` methods fits all of the estimators in the pipeline and return a 
pipeline model of only transformers. Uses data loaded as specified by the data reader to 
generate the initial data set.

In [25]:
val workflow = new OpWorkflow("abc").setResultFeatures(prediction1, cust_density).setReader(trainDataReader)


com.salesforce.op.OpWorkflow@174f33d6

In [26]:
// val workflowModel = workflow.train()
// val workflowModel: OpWorkflowModel = workflow.train()

null

In [27]:
// workflowModel.save(path ="qqq/taxi_0.6.1_spark-2.3.3_cust_density", overwrite = true)

null

In [28]:
// val testDataReader = DataReaders.Simple.avro[Passenger](key = _.passengerId, path = Some("my/test/data/path")) // simply reads and returns data
// val scoredTestData = workflowModel.setReader(testDataReader).score()

null

In [41]:
// val model=workflow.loadModel(path = "/home/whirldata/Documents/moving/from_server/savings/taxi_0.5.3_spark-2.3.3_cust_density/")
val workflowModel= workflow.loadModel(path = "/home/whirldata/Documents/moving/from_server/savings/taxi_0.5.3_spark-2.3.3_cust_density")

com.salesforce.op.OpWorkflowModel@51485629

In [106]:
// val model=OpWorkflow.load(path ="recent_models/taxi_0.5.3_spark-2.3.0")
// // val Model=workflow.load(path ="recent_models/taxi_0.5.3_spark-2.3.0")

null

In [27]:
// val workflowModel = workflow.loadModel("qqq/taxi_0.6.1_spark-2.3.3")//Important.........

com.salesforce.op.OpWorkflowModel@55a31747

In [42]:
val testDataReader = DataReaders.Simple.csvCase[Taxi_prediction](path = Some("/home/whirldata/Documents/moving/testing_tail_data.csv")) 
val scoredTestData = workflowModel.setReader(testDataReader).score()

[key: string, cust_density: double ... 1 more field]

[key: string, cust_density: double ... 1 more field]

In [43]:
val dfScoreAndEvaluate = workflowModel.scoreAndEvaluate(evaluator)
val dfScore = dfScoreAndEvaluate._1.
    withColumnRenamed("",
    "predicted_total_amount")
val dfEvaluate = dfScoreAndEvaluate._2

{
  "RootMeanSquaredError" : 4.512998821812766,
  "MeanSquaredError" : 20.367158365683412,
  "R2" : 0.015603752262764004,
  "MeanAbsoluteError" : 3.9139585980963303
}

In [34]:
println(workflowModel)

In [35]:
workflowModel.summary()

{
  "ModelSelector_000000000013" : {
    "ValidationResults" : [ {
      "ModelName" : "OpLinearRegression_00000000000d_0",
      "ModelParameters" : {
        "elasticNetParam" : 0.1,
        "outputFeatureName" : "OnHour-cust_density-date_col-day_of_the_week-pickup_lat-pickup_long_4-stagesApplied_Prediction_000000000013",
        "inputSchema" : "StructType()",
        "sparkMlStage" : "org.apache.spark.ml.regression.LinearRegression",
        "featuresCol" : "features",
        "outputMetadata" : { },
        "fitIntercept" : true,
        "solver" : "auto",
        "labelCol" : "label",
        "inputFeatures" : [ "{\n  \"name\" : \"cust_density\",\n  \"isResponse\" : true,\n  \"isRaw\" : true,\n  \"uid\" : \"RealNN_00000000001c\",\n  \"typeName\" : \"com.salesforce.op.features.types.RealNN\",\n  \"originFeatures\" : [ \"cust_density\" ],\n  \"stages\" : [ ]\n}", "{\n  \"name\" : \"OnHour-date_col-day_of_the_week-pickup_lat-pickup_long_3-stagesApplied_OPVector_000000000009\",\n  \"

In [36]:
// //Code to save the predicted target column in a separate file
// val dfScoreMod = dfScore.rdd.map(x => x(2).toString.split("->")(1).
//     dropRight(1).dropRight(1))
// dfScoreMod.foreach(println)
// // dfScoreMod.saveAsTextFile("./output/taxi_prediction")
// dfScoreMod.saveAsTextFile("./output/taxi_prediction")

null

### Score and evaluate the model

In [38]:
val (scores, metrics) = workflowModel.scoreAndEvaluate(evaluator)
scores.show(true)

null

In [138]:
val scoredTrainData = workflowModel.score()

[key: string, total_amount: double ... 1 more field]

In [39]:
// //Loading our own test data to predict the column values..............
// val testDataReader = DataReaders.Simple.csvCase[Taxi](path = Some("/home/whirldata/Videos/TransmogrifAI/helloworld/notebooks/testing.csv")) 
// val scoredTestData = workflowModel.setReader(testDataReader).score()

<console>: 126

In [40]:
metrics.toString()

{
  "RootMeanSquaredError" : 4.512998821812766,
  "MeanSquaredError" : 20.367158365683412,
  "R2" : 0.015603752262764004,
  "MeanAbsoluteError" : 3.9139585980963303
}